In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
from pymc import math
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# STA365 HW5

## Part I

In [2]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/My Drive/house-prices-advanced-regression-techniques 2/train.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


From previous homework, I observed that the outcome variable follows an approximately log-normal distribution, so I will make a log transformation. I will also select only some of the numeric variables from the independent variables for the simplicity of this homework.

In [3]:
df1 = df[['LotFrontage', 'LotArea', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'TotRmsAbvGrd', 'SalePrice']]
df1 = df1.dropna()
X = df1.drop('SalePrice', axis = 1)
y = df1['SalePrice']
n, p = X.shape
X = X.values
y = y.values.reshape(-1, 1)
y = np.log(y)
scalerX = StandardScaler()
scalery = StandardScaler()
X = scalerX.fit_transform(X)
y = scalery.fit_transform(y)

In [5]:
print(f"Mean: {np.mean(y)}, Standard Deviation: {np.std(y)}")

Mean: 4.027493483502816e-15, Standard Deviation: 1.0


In [6]:
model = LinearRegression()
model.fit(X, y)
slope = model.coef_
intercept = model.intercept_
print(f"Coefficients: {slope}")
print(f"Intercept: {intercept}")

Coefficients: [[-0.00722875  0.01727701  0.37547416  0.00880168  0.51196367 -0.00485635]]
Intercept: [3.96298626e-15]


Now we proceed with the Bayesian Linear model. I adjusted the hyperparameters based on my observations from the code above.

In [7]:
with pm.Model() as MLR:
  betas = pm.MvNormal('betas', mu=np.zeros(p), cov=np.eye(p), shape=p)
  sigma = pm.TruncatedNormal('sigma', mu=0, sigma=1, lower=0)
  y_est = pm.math.dot(X, betas)
  y_obs = pm.Normal('y_obs', mu=y_est, sigma=sigma, observed=y.reshape(-1))
  idata = pm.sample(chains=4)

/usr/local/lib/python3.10/dist-packages/arviz/utils.py:184: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  numba_fn = numba.jit(**self.kwargs)(self.function)


## Part II

1. $p(y|\beta, \sigma^2I, X)=(2\pi)^{-n/2}\exp(-\frac{1}{2\sigma^2}(y-X\beta)^T(y-X\beta)) \\ ∝\exp(\frac{1}{\sigma^2}\beta^TX^Ty-\frac{1}{2\sigma^2}\beta^TX^TX\beta)\\ ∝\exp(-\frac{1}{2}(\beta-(X^TX)^{-1}X^Ty\sigma^{-4})^T(X^TX\sigma^{-2})(\beta-(X^TX)^{-1}X^Ty\sigma^{-4}))$

2. $E[\beta|\Sigma, X, y]=Var[\beta|\Sigma, X, y]^{-1}(X^T\Sigma^{-1}y+\Sigma_{\beta}^{-1}\beta_0) \\ = \{[X^T\Sigma^{-1}X]^{-1}+\Sigma_{\beta}^{-1}\}^{-1}(X^T\Sigma^{-1}y+\Sigma_{\beta}^{-1}\beta_0)$

3. From the formula in part 2, if $\Sigma=I$ and $\Sigma_{\beta}=0$, then $E[\beta|\Sigma, X, y]=(X^TX)^{-1}X^Ty$

4. Since $E[\hat{y}=X\beta|\Sigma,X,y]=E(X)E[\beta|\Sigma,X,y]=X\cdot E[\beta|\Sigma,X,y]$, from the result in question 3, if $\Sigma=I$ and $\Sigma_{\beta}=0$, then $E[\hat{y}=X\beta|\Sigma,X,y]=X\cdot E[\beta|\Sigma,X,y]=X\cdot(X^TX)^{-1}X^Ty$

5. We know from lecture that $Var[\beta|\Sigma,X,y]=[X^T\Sigma^{-1}X]^{-1}+\Sigma_{\beta}^{-1}$, so in the case of $\Sigma=I$ and $\Sigma_{\beta}=0$, $Var[\beta|\Sigma,X,y]=(X^TX)^{-1}$

## Part III

We will use the same house pricing data in part 1. For $\mu$, a $MVN$ prior with mean $0$ and covariance $I$ would be a very sensible choice because I standardized the data in the preprocessing.

In [ ]:
# We have the preprocessed X and y from part I
with pm.Model() as MVN_LKJ:
    packed_L = pm.LKJCholeskyCov("packed_L", n=p, eta=2.0,
                                 sd_dist=pm.Exponential.dist(1.0, shape=2), compute_corr=False)
    L = pm.expand_packed_triangular(p, packed_L)
    # Sigma = pm.Deterministic('Sigma', L.dot(L.T)) # Don't use a covariance matrix parameterization
    mu = pm.MvNormal('mu', mu=np.array(0), cov=np.eye(p), shape=p);
    # y = pm.MvNormal('y', mu=mu, cov=Sigma, shape=(n,1), observed=y)
    # Figure out how to parameterize this with a Cholesky factor to improve computational efficiency
    y_obs = pm.MvNormal('y_obs', mu=mu, chol=L, observed=y)

with MVN_LKJ:
    idata = pm.sample()